## 特征工程

In [1]:
# 数据和特征决定了机器学习的上限，而模型和算法只是逼近这个上限而已。 ⚠️重点！数据的数量和质量比算法重要的多！
# 特征工程包括：
# 1.特征使用 （关注点都在特征的原数据上）
    # 数据的选择：是分析和我们目标最相关的数据都有哪些，这些数据如何获取。
    # 数据的可用性判断：是指数据特征是否可以持续输出
# 2.特征获取
    # 特征来源：即我们需要的特征来自与哪张表或哪个文件
    # 特征存储：如特征来自于不同的文件或不同的数据库，就要将数据进行规整，存储在将来想要使用的媒介中
# 3.特征处理：目的是数据属性和特征尽可能大的发挥作用，体现差别
    # 数据清洗：
    # 特征预处理：
# 4.特征监控
    # 现有特征：主要看是否对现有的任务还存在积极的作用
    # 新的特征：看知否有助于提高效果，或更能代表我们的任务目标

#### 数据清洗

In [2]:
# 数据样本抽样
# 样本要具有代表性
# 样本比例平衡以及样本不平衡时如何处理
# 考虑全量数据 （hadoop、spark大数据工具）

In [3]:
# 异常值（空值）处理
# 识别异常值和重复值  pandas:isnull()/duplicated()
# 直接丢弃（包括重复数据） pandas:drop()/dropna()/drop_duplicated()
# 将异常值当作一个新的属性，替代原值 pandas:fillna()
# 集中值指代：pandas:fillna() 集中值可以是：除异常值外的均值、中位数、众数等
# 边界值指代：pandas:fillna() 利用四分位数的边界
# 插值： pandas:interpolate() --- Series

In [2]:
# 字符类型的空值是：None
# 数值类型的空值是：NaN
df = [[],[]]
# 查看空值
df.isnull()
# 删除空值
df.dropna()
# 去掉B行的空值
df.dropna(subset=["B"])
# 识别重复值
df.duplicated(["A"])
# 删除重复值
# keep="first" last, False:删除所有重复值
# inplace=True  可以让DF的index也跟着发生变化
df.drop_duplicates(["A"], keep="first")
# 标注异常值
df.fillna("b*")
# 集中值指代 均值
df.fillna(df["E"].mean())
# 插值 （只能用于series数据），返回的是相邻两个值的平均值，处于两端的话，就取最近的数
# method="spline", order=3  三次样条
df["E"].interpolate()
# 用四分位数确定上下界的方法进行过滤
upper_q = df["D"].quantile(0.75)
lower_q = df["D"].quantile(0.25)
q_int = upper_q - lower_q
k = 1.5
df[df["D"]>lower_q-k*q_int][df["D"]<upper_q+k*q_int]
# 去除异常值
df.drop(2)
# 或者
df[[True if item.startswith("f") else False for item in list(df["F"].values)]]

AttributeError: 'list' object has no attribute 'isnull'

#### 标注

In [3]:
# 标注（标记、标签、label）

#### 特征预处理

In [5]:
# 特征选择：剔除与标注不相关或者冗余的特征，减少特征的个数。作用：减少了训练的时间，减少了过拟合。是数据规约的一种处理方式（另一个为抽样）
    # 特征选择有三个切入思路；
    # 1.过滤思想：就是直接评价某个特征与标注的相关性的特征，如果与标注的相关性非常小就去掉。（复习下面的特征选择表）
    # 2.包裹思想：遍历特征子集。假设所有的集合是一个集合X，最佳的特征组合是它的一个子集，我们的任务就是要找到这个子集。
    # RFE算法：
    # 第一步：列出集合X。
    # 第二步：构造简单的模型进行训练，根据系数去掉比较弱的特征。
    # 第三部：余下的特征重复这个过程，直到评价指标下降较大或者低于阀值，停止
    # 3.嵌入思想：建立简单的回归模型。根据一个简单的模型来分析特征的重要性。最常用的方式是用正则化的方式来做特征选择。
# 特征变换：对指化、离散化、数据平滑、归一化（标准化）、数值化、正规化
# 特征降维 
# 特征衍生 

#### 特征选择表

In [6]:
#    数据类型                           可用方法
# 连续----连续                       相关系数、假设检验
# 连续----离散（二值）                相关系数、连续二值化（最小Gini切分，最大熵增益切分）
# 连续----离散（非二值）              相关系数（定序）
# 离散（二值）----离散（二值）         相关系数、熵相关、F分值
# 离散----离散（非二值）              熵相关、Gini、相关系数（定序）

#### 代码实现

In [7]:
import numpy as np
import pandas as pd
import scipy.stats as ss

In [16]:
# 生成一组数据
df = pd.DataFrame({"A":ss.norm.rvs(size=10),
               "B":ss.norm.rvs(size=10),
               "C":ss.norm.rvs(size=10),
               "D":np.random.randint(low=0, high=2, size=10)})

In [17]:
df

,A,B,C,D
0,0.068055,-0.443143,1.108843,1
1,1.660826,-0.154182,-1.728332,1
2,0.075818,-0.223309,0.407641,1
3,-1.089937,-0.967766,-1.140794,1
4,-1.022013,0.151264,-0.160691,1
5,0.086538,0.258648,-1.856277,0
6,-1.424559,-0.083167,1.027530,1
7,-0.627077,1.153571,-1.524347,0
8,-0.850279,-0.743631,0.424328,0
9,-0.427147,-1.664223,1.348109,1


In [18]:
# 引入SVR回归器、
# DecisionTreeRegressor决策树回归器
from sklearn.svm import SVR
from sklearn.tree import DecisionTreeRegressor

In [21]:
# 确定特征和标注
# 特征
X = df.loc[:,["A", "B", "C"]]
# 标注
Y = df.loc[:, "D"]

In [23]:
# 特征选择常用的三个类
# 过滤思想 SelectKBest
# 包裹思想 RFE
# 嵌入思想 SelectFromModel
from sklearn.feature_selection import SelectKBest, RFE, SelectFromModel

#### 过滤思想

In [25]:
# f_classif 通过方差分析的F值进行判定的（默认的）
# mutual_info_classif 互信息
# chi2 卡方校验
skb = SelectKBest(k=2)
skb.fit(X,Y)
skb.transform(X)

array([[-0.44314267,  1.10884264],
       [-0.15418172, -1.72833239],
       [-0.22330872,  0.40764105],
       [-0.96776614, -1.14079445],
       [ 0.15126358, -0.16069073],
       [ 0.25864802, -1.85627674],
       [-0.08316678,  1.02752952],
       [ 1.15357136, -1.52434747],
       [-0.74363087,  0.424328  ],
       [-1.66422293,  1.3481086 ]])

#### 包裹思想 RFE

In [26]:
# SVR 线性回归器 
# n_features_to_select 最终要选择的特征数
# step 每一步要去掉多少个特征  step=1每迭代一次去掉一个特征 
rfe = RFE(estimator=SVR(kernel="linear"), n_features_to_select=2, step=1)
# fit_transform 拟合过后再进行变换
rfe.fit_transform(X, Y)

array([[ 0.06805487, -0.44314267],
       [ 1.66082604, -0.15418172],
       [ 0.07581797, -0.22330872],
       [-1.08993669, -0.96776614],
       [-1.02201321,  0.15126358],
       [ 0.08653789,  0.25864802],
       [-1.42455855, -0.08316678],
       [-0.62707671,  1.15357136],
       [-0.85027892, -0.74363087],
       [-0.42714716, -1.66422293]])

#### 嵌入思想 SelectFromModel

In [27]:
# threshold 表示重要性因子的数（低于多少就会被去掉）
sfm = SelectFromModel(estimator=DecisionTreeRegressor(), threshold=0.1)
sfm.fit_transform(X, Y)

array([[-0.44314267],
       [-0.15418172],
       [-0.22330872],
       [-0.96776614],
       [ 0.15126358],
       [ 0.25864802],
       [-0.08316678],
       [ 1.15357136],
       [-0.74363087],
       [-1.66422293]])

### 特征变换

In [28]:
# 特征变换：就是根据特征的特性进行一定方式的转换，使特征能够发挥出它的特点
# 特征变换的方法：
# 1.对指化：对数据进行对数化和指数化的过程  使用函数 softmax 即可完成  numpy.exp
# 2.对数化：对数据取对数的过程，底可以取e、10、2  例如月收入情况  numpy.log

### 离散化

In [29]:
# 将连续变量分成几段（bins）